# Distributed Data Systems Project Group 12
### Task 2 - Data Preprocessing and Writing/Reading to HDFS

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row, SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, Bucketizer

from datetime import datetime
import dateutil

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1579045070657_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read in Argo data as Spark DataFrame

In [3]:
# argo_schema = StructType([StructField("profile_id", StringType(), True),
#                           StructField("pres", DoubleType(), True),
#                           StructField("temp", DoubleType(), True),
#                           StructField("lat", DoubleType(), True),
#                           StructField("lon", DoubleType(), True),
#                           StructField("psal", DoubleType(), True),
#                           StructField("date", TimestampType(), True),
#                         ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
argo_df_og = ss.read.csv("s3://msds-argo-clustering/argo_data.csv", header=True, inferSchema=True)

# Do it this way, because all nulls if define schema ahead of item

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
argo_df_og.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- profile_id: string (nullable = true)
 |-- pres: double (nullable = true)
 |-- temp: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- psal: string (nullable = true)
 |-- date: timestamp (nullable = true)

In [6]:
# Cast temp as DoubleType()
argo_df_og = argo_df_og.withColumn("tempTmp", argo_df_og['temp'].cast(DoubleType()))\
                       .drop("temp")\
                       .withColumnRenamed("tempTmp", "temp")\
                       .select("profile_id", "pres", "temp", "lat", "lon", "psal", "date")\
                       .persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
argo_df_og.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+-----+-------+--------+----+--------------------+
| profile_id|pres| temp|    lat|     lon|psal|                date|
+-----------+----+-----+-------+--------+----+--------------------+
|5904437_131| 6.7|8.531|-49.507|-177.884|NULL|2018-06-01 23:57:...|
|5904437_131|10.1| 8.53|-49.507|-177.884|NULL|2018-06-01 23:57:...|
|5904437_131|20.1|8.531|-49.507|-177.884|NULL|2018-06-01 23:57:...|
|5904437_131|31.0|8.527|-49.507|-177.884|NULL|2018-06-01 23:57:...|
|5904437_131|41.0|8.528|-49.507|-177.884|NULL|2018-06-01 23:57:...|
+-----------+----+-----+-------+--------+----+--------------------+
only showing top 5 rows

## Filter DataFrame by conditions to keep records

In [8]:
argo_filterby = argo_df_og.groupBy("profile_id") \
                          .agg(min("pres").alias("min_pres"), 
                               max("pres").alias("max_pres"), 
                               count("profile_id").alias("count_profile_id"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
argo_filterby.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+--------+----------------+
| profile_id|min_pres|max_pres|count_profile_id|
+-----------+--------+--------+----------------+
|3901186_158|    2.83| 2017.53|            1009|
|5903298_277|     6.4|  2000.5|              71|
| 7900680_19|    1.04|  2003.8|            1008|
|5904537_156|    1.08|  1446.0|             729|
| 5905053_43|     4.9|  2001.1|             115|
+-----------+--------+--------+----------------+
only showing top 5 rows

In [10]:
# Now, here are the profile_ids we want to keep, to be inner joined with original argo_df_og
argo_keep_ids = argo_filterby.filter("count_profile_id >= 50 and min_pres <= 25 and max_pres >= 999") \
                             .select("profile_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Expect the number profile_ids to keep < the total number of profile_ids originally
print(argo_keep_ids.count())
print(argo_df_og.select("profile_id").distinct().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

98525
103913

In [12]:
# Inner join the profile_ids to keep with original argo_df_og to filter and keep only desired IDs
argo_df_keep = argo_keep_ids.join(argo_df_og, "profile_id", "inner").persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
argo_df_keep.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+-----+-------+-------+------+-------------------+
| profile_id|pres| temp|    lat|    lon|  psal|               date|
+-----------+----+-----+-------+-------+------+-------------------+
|1900977_343|10.0|8.954|-55.157|178.214|34.123|2019-02-19 04:32:51|
|1900977_343|16.5|8.954|-55.157|178.214|34.123|2019-02-19 04:32:51|
|1900977_343|23.4|8.953|-55.157|178.214|34.125|2019-02-19 04:32:51|
|1900977_343|30.0|8.952|-55.157|178.214|34.126|2019-02-19 04:32:51|
|1900977_343|37.3|8.933|-55.157|178.214|34.126|2019-02-19 04:32:51|
+-----------+----+-----+-------+-------+------+-------------------+
only showing top 5 rows

In [14]:
# Expect argo_df_keep.count() < argo_df.count()
print(argo_df_keep.count())
print(argo_df_og.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

62117082
63590252

In [15]:
argo_df = argo_df_keep.select("profile_id", "pres", "temp", "lat", "lon", "psal", "date", 
                              month("date").alias("month"), year("date").alias("year")) \
                      .persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
argo_df.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+-----+-------+-------+------+-------------------+-----+----+
| profile_id|pres| temp|    lat|    lon|  psal|               date|month|year|
+-----------+----+-----+-------+-------+------+-------------------+-----+----+
|1900977_343|10.0|8.954|-55.157|178.214|34.123|2019-02-19 04:32:51|    2|2019|
+-----------+----+-----+-------+-------+------+-------------------+-----+----+
only showing top 1 row

## Bucketing pressure levels

In [17]:
bin_range = [-5, 0,  5,  10,  15,  20,  25,  30,  35,  40,  45,  50,  55,  60,  65,
        70,  75,  80,  85,  90,  95, 100, 105, 110, 115, 120, 125, 130,
       135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195,
       200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260,
       265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325,
       330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390,
       395, 400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455,
       460, 465, 470, 475, 480, 485, 490, 495, 500, 505, 510, 515, 520,
       525, 530, 535, 540, 545, 550, 555, 560, 565, 570, 575, 580, 585,
       590, 595, 600, 605, 610, 615, 620, 625, 630, 635, 640, 645, 650,
       655, 660, 665, 670, 675, 680, 685, 690, 695, 700, 705, 710, 715,
       720, 725, 730, 735, 740, 745, 750, 755, 760, 765, 770, 775, 780,
       785, 790, 795, 800, 805, 810, 815, 820, 825, 830, 835, 840, 845,
       850, 855, 860, 865, 870, 875, 880, 885, 890, 895, 900, 905, 910,
       915, 920, 925, 930, 935, 940, 945, 950, 955, 960, 965, 970, 975,
       980, 985, 990, 995, float("inf")]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
bucketizer = Bucketizer(splits=bin_range,inputCol="pres", outputCol="pres_buckets")
argo_df_buck = bucketizer.setHandleInvalid("keep").transform(argo_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
argo_df_buck.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+-----+-------+-------+------+-------------------+-----+----+------------+
| profile_id|pres| temp|    lat|    lon|  psal|               date|month|year|pres_buckets|
+-----------+----+-----+-------+-------+------+-------------------+-----+----+------------+
|1900977_343|10.0|8.954|-55.157|178.214|34.123|2019-02-19 04:32:51|    2|2019|         3.0|
|1900977_343|16.5|8.954|-55.157|178.214|34.123|2019-02-19 04:32:51|    2|2019|         4.0|
|1900977_343|23.4|8.953|-55.157|178.214|34.125|2019-02-19 04:32:51|    2|2019|         5.0|
|1900977_343|30.0|8.952|-55.157|178.214|34.126|2019-02-19 04:32:51|    2|2019|         7.0|
|1900977_343|37.3|8.933|-55.157|178.214|34.126|2019-02-19 04:32:51|    2|2019|         8.0|
+-----------+----+-----+-------+-------+------+-------------------+-----+----+------------+
only showing top 5 rows

## Save to HDFS

In [20]:
argo_df_buck.write.save('hdfs:///argo', format='parquet', mode='append')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read HDFS back in

In [21]:
sqlContext = SQLContext(sc)
argo_df_buck_reread = sqlContext.read.format('parquet').load('hdfs:///argo') 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
argo_df_buck_reread.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----+-----+---------+---------+------+--------------------+-----+----+------------+
| profile_id| pres| temp|      lat|      lon|  psal|                date|month|year|pres_buckets|
+-----------+-----+-----+---------+---------+------+--------------------+-----+----+------------+
|1901725_129| 1.04|7.564|-51.47012|-77.56111|33.668|2018-07-19 11:46:...|    7|2018|         1.0|
|1901725_129| 1.96|7.564|-51.47012|-77.56111|33.668|2018-07-19 11:46:...|    7|2018|         1.0|
|1901725_129| 3.04|7.564|-51.47012|-77.56111|33.668|2018-07-19 11:46:...|    7|2018|         1.0|
|1901725_129|  4.0|7.564|-51.47012|-77.56111|33.668|2018-07-19 11:46:...|    7|2018|         1.0|
|1901725_129|  5.0|7.564|-51.47012|-77.56111|33.668|2018-07-19 11:46:...|    7|2018|         2.0|
|1901725_129|  6.0|7.564|-51.47012|-77.56111|33.668|2018-07-19 11:46:...|    7|2018|         2.0|
|1901725_129| 7.08|7.565|-51.47012|-77.56111|33.668|2018-07-19 11:46:...|    7|2018|         2.0|
|1901725_129|  8.0|7